<a href="https://colab.research.google.com/github/JIJASH/data_mining_and_warehousing/blob/main/lab_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab IV: Clustering and Outlier Detection Algorithms

Implementation of clustering and outlier detection algorithms from scratch.

In [1]:
# Import libraries and utility functions
import math
import random
import csv
from collections import defaultdict

def euclidean_distance(p1, p2):
    return math.sqrt(sum((x - y) ** 2 for x, y in zip(p1, p2)))

def read_csv(filename):
    data = []
    with open(filename, 'r') as file:
        reader = csv.reader(file)
        next(reader)  # Skip header
        for row in reader:
            data.append([float(x) for x in row])
    return data

In [2]:
data1 = read_csv('/content/drive/MyDrive/data_mining_and_warehousing/data.csv')      # For algorithms 1-5
data2 = read_csv('/content/drive/MyDrive/data_mining_and_warehousing/data2.csv')     # For DBSCAN
data3 = read_csv('/content/drive/MyDrive/data_mining_and_warehousing/data3.csv')     # For outlier analysis
data4 = read_csv('/content/drive/MyDrive/data_mining_and_warehousing/data4.csv')     # For mini-batch k-means

In [3]:
# 1. K-Means Algorithm
def kmeans(data, k, max_iterations=100):
    n_features = len(data[0])

    # Initialize centroids randomly
    min_vals = [min(point[i] for point in data) for i in range(n_features)]
    max_vals = [max(point[i] for point in data) for i in range(n_features)]
    centroids = [[random.uniform(min_vals[i], max_vals[i]) for i in range(n_features)] for _ in range(k)]

    for iteration in range(max_iterations):
        # Assign points to closest centroid
        clusters = [[] for _ in range(k)]
        for point in data:
            distances = [euclidean_distance(point, centroid) for centroid in centroids]
            closest_centroid = distances.index(min(distances))
            clusters[closest_centroid].append(point)

        # Update centroids
        new_centroids = []
        for cluster in clusters:
            if cluster:
                centroid = [sum(point[i] for point in cluster) / len(cluster) for i in range(n_features)]
                new_centroids.append(centroid)
            else:
                new_centroids.append(centroids[clusters.index(cluster)])

        # Check convergence
        if all(euclidean_distance(old, new) < 1e-6 for old, new in zip(centroids, new_centroids)):
            break
        centroids = new_centroids

    return centroids, clusters

# Test on data.csv
centroids, clusters = kmeans(data1, 3)
print("K-Means centroids:", centroids)
print("Cluster sizes:", [len(cluster) for cluster in clusters])

K-Means centroids: [[1.8251254225611786, 2.291875885218977], [7.0, 1.0], [5.882762642208976, 5.744872196239354]]
Cluster sizes: [11, 1, 10]


In [4]:
# 2. K-Means++ Algorithm
def kmeans_plus_plus(data, k, max_iterations=100):
    n_features = len(data[0])
    centroids = [data[random.randint(0, len(data) - 1)]]

    # Choose remaining centroids using K-means++ method
    for _ in range(1, k):
        distances = []
        for point in data:
            min_dist = min(euclidean_distance(point, centroid) for centroid in centroids)
            distances.append(min_dist ** 2)

        total_dist = sum(distances)
        probabilities = [d / total_dist for d in distances]

        r = random.random()
        cumulative_prob = 0
        for i, prob in enumerate(probabilities):
            cumulative_prob += prob
            if r <= cumulative_prob:
                centroids.append(data[i])
                break

    # Run standard K-means
    for iteration in range(max_iterations):
        clusters = [[] for _ in range(k)]
        for point in data:
            distances = [euclidean_distance(point, centroid) for centroid in centroids]
            closest_centroid = distances.index(min(distances))
            clusters[closest_centroid].append(point)

        new_centroids = []
        for cluster in clusters:
            if cluster:
                centroid = [sum(point[i] for point in cluster) / len(cluster) for i in range(n_features)]
                new_centroids.append(centroid)
            else:
                new_centroids.append(centroids[clusters.index(cluster)])

        if all(euclidean_distance(old, new) < 1e-6 for old, new in zip(centroids, new_centroids)):
            break
        centroids = new_centroids

    return centroids, clusters

# Test on data.csv
centroids_pp, clusters_pp = kmeans_plus_plus(data1, 3)
print("K-Means++ centroids:", centroids_pp)
print("Cluster sizes:", [len(cluster) for cluster in clusters_pp])

K-Means++ centroids: [[1.8251254225611786, 2.291875885218977], [5.882762642208976, 5.744872196239354], [7.0, 1.0]]
Cluster sizes: [11, 10, 1]


In [5]:
# 3. K-Medoids Algorithm
def k_medoids(data, k, max_iterations=100):
    medoids = random.sample(data, k)

    for iteration in range(max_iterations):
        clusters = [[] for _ in range(k)]
        for point in data:
            distances = [euclidean_distance(point, medoid) for medoid in medoids]
            closest_medoid = distances.index(min(distances))
            clusters[closest_medoid].append(point)

        new_medoids = []
        for i, cluster in enumerate(clusters):
            if not cluster:
                new_medoids.append(medoids[i])
                continue

            min_total_dist = float('inf')
            best_medoid = medoids[i]
            for candidate in cluster:
                total_dist = sum(euclidean_distance(candidate, point) for point in cluster)
                if total_dist < min_total_dist:
                    min_total_dist = total_dist
                    best_medoid = candidate
            new_medoids.append(best_medoid)

        if all(old == new for old, new in zip(medoids, new_medoids)):
            break
        medoids = new_medoids

    return medoids, clusters

# Test on data.csv
medoids, clusters_medoids = k_medoids(data1, 3)
print("K-Medoids medoids:", medoids)
print("Cluster sizes:", [len(cluster) for cluster in clusters_medoids])

K-Medoids medoids: [[6.0472697432815465, 5.00267626965058], [1.882923312638332, 1.8829315215254097], [5.1943044958043885, 6.26298861284197]]
Cluster sizes: [6, 10, 6]


In [6]:
# 4. Agglomerative Hierarchical Clustering
def agglomerative_clustering(data, n_clusters):
    clusters = [[point] for point in data]

    while len(clusters) > n_clusters:
        min_distance = float('inf')
        merge_i, merge_j = 0, 1

        for i in range(len(clusters)):
            for j in range(i + 1, len(clusters)):
                dist = min(euclidean_distance(p1, p2) for p1 in clusters[i] for p2 in clusters[j])
                if dist < min_distance:
                    min_distance = dist
                    merge_i, merge_j = i, j

        merged_cluster = clusters[merge_i] + clusters[merge_j]
        new_clusters = [clusters[i] for i in range(len(clusters)) if i != merge_i and i != merge_j]
        new_clusters.append(merged_cluster)
        clusters = new_clusters

    return clusters

# Test on data.csv
agg_clusters = agglomerative_clustering(data1, 3)
print("Agglomerative cluster sizes:", [len(cluster) for cluster in agg_clusters])

Agglomerative cluster sizes: [1, 10, 11]


In [7]:
# 5. Divisive Clustering
def divisive_clustering(data, n_clusters):
    clusters = [data]

    while len(clusters) < n_clusters:
        max_variance = -1
        split_index = 0

        for i, cluster in enumerate(clusters):
            if len(cluster) <= 1:
                continue

            n_features = len(cluster[0])
            variance = 0
            for feature in range(n_features):
                feature_values = [point[feature] for point in cluster]
                mean_val = sum(feature_values) / len(feature_values)
                variance += sum((val - mean_val) ** 2 for val in feature_values)

            if variance > max_variance:
                max_variance = variance
                split_index = i

        cluster_to_split = clusters[split_index]
        if len(cluster_to_split) <= 1:
            break

        sub_centroids, sub_clusters = kmeans(cluster_to_split, 2, max_iterations=50)
        new_clusters = [clusters[i] for i in range(len(clusters)) if i != split_index]
        new_clusters.extend(sub_clusters)
        clusters = new_clusters

    return clusters

# Test on data.csv
div_clusters = divisive_clustering(data1, 3)
print("Divisive cluster sizes:", [len(cluster) for cluster in div_clusters])

Divisive cluster sizes: [11, 10, 1]


In [8]:
# 6. DBSCAN Algorithm
def dbscan(data, eps, min_pts):
    n_points = len(data)
    labels = [-1] * n_points  # -1: unclassified, -2: noise
    cluster_id = 0

    def get_neighbors(point_idx):
        neighbors = []
        for i in range(n_points):
            if euclidean_distance(data[point_idx], data[i]) <= eps:
                neighbors.append(i)
        return neighbors

    def expand_cluster(point_idx, neighbors, cluster_id):
        labels[point_idx] = cluster_id
        i = 0
        while i < len(neighbors):
            neighbor_idx = neighbors[i]
            if labels[neighbor_idx] == -1:
                labels[neighbor_idx] = cluster_id
                new_neighbors = get_neighbors(neighbor_idx)
                if len(new_neighbors) >= min_pts:
                    neighbors.extend(new_neighbors)
            elif labels[neighbor_idx] == -2:
                labels[neighbor_idx] = cluster_id
            i += 1

    for point_idx in range(n_points):
        if labels[point_idx] != -1:
            continue
        neighbors = get_neighbors(point_idx)
        if len(neighbors) < min_pts:
            labels[point_idx] = -2
        else:
            expand_cluster(point_idx, neighbors, cluster_id)
            cluster_id += 1

    clusters = defaultdict(list)
    noise_points = []
    for i, label in enumerate(labels):
        if label == -2:
            noise_points.append(data[i])
        else:
            clusters[label].append(data[i])

    return dict(clusters), noise_points

# Test on data2.csv
clusters_dbscan, noise_points = dbscan(data2, 0.5, 3)
print("DBSCAN clusters:", len(clusters_dbscan))
print("Noise points:", len(noise_points))

DBSCAN clusters: 1
Noise points: 3


In [9]:
# 7. Z-Score Outlier Detection
def z_score_outlier_detection(data, threshold=2.0):
    values = [point[0] for point in data] if isinstance(data[0], list) else data
    n = len(values)

    mean = sum(values) / n
    variance = sum((x - mean) ** 2 for x in values) / n
    std_dev = math.sqrt(variance)

    outliers = []
    for i, value in enumerate(values):
        z_score = (value - mean) / std_dev if std_dev != 0 else 0
        if abs(z_score) > threshold:
            outliers.append((i, value))

    return outliers

# Test on data3.csv
outliers_zscore = z_score_outlier_detection(data3)
print("Z-Score outliers:", len(outliers_zscore))

Z-Score outliers: 1


In [10]:
# 8. IQR Outlier Detection
def iqr_outlier_detection(data):
    values = [point[0] for point in data] if isinstance(data[0], list) else data
    sorted_values = sorted(values)
    n = len(sorted_values)

    def get_quartile(sorted_list, quartile):
        index = (len(sorted_list) - 1) * quartile
        if index.is_integer():
            return sorted_list[int(index)]
        else:
            lower = sorted_list[int(index)]
            upper = sorted_list[int(index) + 1]
            return lower + (upper - lower) * (index - int(index))

    q1 = get_quartile(sorted_values, 0.25)
    q3 = get_quartile(sorted_values, 0.75)
    iqr = q3 - q1

    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr

    outliers = []
    for i, value in enumerate(values):
        if value < lower_bound or value > upper_bound:
            outliers.append((i, value))

    return outliers

# Test on data3.csv
outliers_iqr = iqr_outlier_detection(data3)
print("IQR outliers:", len(outliers_iqr))

IQR outliers: 2


In [11]:
# 9. Mini-Batch K-Means Algorithm
def mini_batch_kmeans(data, k, batch_size=10, max_iterations=100):
    n_features = len(data[0])

    min_vals = [min(point[i] for point in data) for i in range(n_features)]
    max_vals = [max(point[i] for point in data) for i in range(n_features)]
    centroids = [[random.uniform(min_vals[i], max_vals[i]) for i in range(n_features)] for _ in range(k)]
    counts = [0] * k

    for iteration in range(max_iterations):
        batch = random.sample(data, min(batch_size, len(data)))

        for point in batch:
            distances = [euclidean_distance(point, centroid) for centroid in centroids]
            closest_idx = distances.index(min(distances))
            counts[closest_idx] += 1
            learning_rate = 1.0 / counts[closest_idx]

            for i in range(n_features):
                centroids[closest_idx][i] = (1 - learning_rate) * centroids[closest_idx][i] + learning_rate * point[i]

    clusters = [[] for _ in range(k)]
    for point in data:
        distances = [euclidean_distance(point, centroid) for centroid in centroids]
        closest_centroid = distances.index(min(distances))
        clusters[closest_centroid].append(point)

    return centroids, clusters

# Test on data4.csv
centroids_mb, clusters_mb = mini_batch_kmeans(data4, 4, 20)
print("Mini-Batch K-Means centroids:", centroids_mb)
print("Cluster sizes:", [len(cluster) for cluster in clusters_mb])

Mini-Batch K-Means centroids: [[3.675556769339268, -5.730389523141737], [-6.776849326483863, -6.798302047358834], [4.614522019219613, 2.0748345517529203], [-2.672233536377532, 8.937587930983076]]
Cluster sizes: [0, 66, 67, 67]
